In [ ]:
!pip install tensorflow_decision_forests -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.5 MB/s eta 0:00:00


In [44]:
# imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import tensorflow_decision_forests as tfdf

drive.mount('/content/drive')
titanic_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/titanic-test.csv")
titanic_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/titanic-train.csv")

# pré-processamento

# name - normalização de texto (letras minúsculas), tokenização; ticket - separação das partes

# preprocess() https://www.kaggle.com/code/gusthema/titanic-competition-w-tensorflow-decision-forests
# def preprocess(df):
#   df = df.copy()
#   def tokenize_name(name):
#     name = name.lower()
#     return " ".join([one_name.strip(",()[].\"'") for one_name in name.split(" ")])

#   def ticket_number(x):
#     return x.split(" ")[-1]

#   def ticket_item(x):
#     items = x.split(" ")
#     if len(items) == 1:
#       return "NONE"
#     return "_".join(items[0:-1])
#   df["Name"] = df["Name"].apply(tokenize_name)
#   df["Ticket_number"] = df["Ticket"].apply(ticket_number)
#   df["Ticket_item"] = df["Ticket"].apply(ticket_item)
#   return df

# titanic_train = preprocess(titanic_train)
# titanic_test = preprocess(titanic_test)

# sex - vamos converter para um valor numérico
# 0 - female, 1 - male
titanic_train['Sex'] = LabelEncoder().fit_transform(titanic_train['Sex'])
titanic_test['Sex'] = LabelEncoder().fit_transform(titanic_test['Sex'])

# cabin - tem muitos valores faltantes; solução escolhida: exclusão da coluna
titanic_train = titanic_train.drop('Cabin', axis=1)
titanic_test = titanic_test.drop('Cabin', axis=1)

# embarked - deletamos as linhas em que há valor faltante em 'Embarked'
titanic_train.dropna(subset=["Embarked"])
titanic_test.dropna(subset=["Embarked"])

# após isso, usamos a técnica de one hot encoded
titanic_train = pd.concat([titanic_train, pd.get_dummies(titanic_train['Embarked'], prefix='Embarked')], axis=1)
titanic_train = titanic_train.drop('Embarked', axis=1)
titanic_test = pd.concat([titanic_test, pd.get_dummies(titanic_test['Embarked'], prefix='Embarked')], axis=1)
titanic_test = titanic_test.drop('Embarked', axis=1)

# age - substituímos os valores faltantes pelo valor da média de todas as idades
titanic_train.fillna(titanic_train['Age'].median(), inplace=True)
titanic_test.fillna(titanic_test['Age'].median(), inplace=True)

'''
treinamento do modelo

Ideia: criar um ensemble, que é o uso de vários modelos
Vários modelos são treinados e suas previsões são armazenadas; ao fim, as previsões são combinadas
para produção de um resultado final robusto
'''

from sklearn.ensemble import RandomForestClassifier
import numpy as np

titanic_train_prep = titanic_train.drop('Survived', axis=1)
titanic_train_prep = titanic_train_prep.drop('PassengerId', axis=1)
titanic_train_prep = titanic_train_prep.drop('Name', axis=1)
titanic_train_prep = titanic_train_prep.drop('Ticket', axis=1)
titanic_train_labels = titanic_train['Survived'].copy()

titanic_test_model = titanic_test.drop('PassengerId', axis=1)
titanic_test_model = titanic_test_model.drop('Name', axis=1)
titanic_test_model = titanic_test_model.drop('Ticket', axis=1)

pred = None
num_pred = 0

for i in range(100):
  random_f = RandomForestClassifier(random_state=i)
  random_f.fit(titanic_train_prep, titanic_train_labels)
  # previsão do conjunto de teste
  pred_test = random_f.predict(titanic_test_model)
  # soma das previsões
  if pred is None:
      pred = pred_test
  else:
      pred += pred_test
  num_pred += 1

pred = pred.astype(float)
pred /= 100.0
pred

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array([0.  , 0.  , 0.  , 1.  , 0.41, 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 1.  , 1.  , 0.  , 1.  , 0.02, 0.75, 1.  , 0.84,
       1.  , 0.95, 1.  , 0.  , 1.  , 1.  , 0.99, 0.  , 0.  , 0.  , 0.96,
       0.  , 0.99, 0.96, 0.92, 0.  , 0.  , 1.  , 0.  , 1.  , 0.  , 1.  ,
       1.  , 0.  , 0.05, 0.  , 1.  , 0.95, 0.27, 0.  , 1.  , 1.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.92, 1.  ,
       1.  , 0.  , 0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
       1.  , 0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ,
       1.  , 1.  , 0.  , 0.48, 1.  , 0.  , 0.07, 0.  , 1.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 0.  , 0.  , 0.94, 0.  , 0.  , 0.  , 1.  , 0.  ,
       0.  , 1.  , 1.  , 0.26, 1.  , 0.  , 0.  , 1.  , 0.06, 1.  , 1.  ,
       0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 1.  , 0.05, 0.  , 1.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ,
       0.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 1.

In [46]:
my_submission = pd.DataFrame({'PassengerId': titanic_test["PassengerId"], 'Survived': (pred >= 0.5).astype(int)})
my_submission.to_csv('LudmilaSubmission2.csv', index=False)
# kaggle score: 0.76315